## Goal
- I want to pull up the comments that relates to the user's preferences.
- As an extension, I hope to highlight the words that relates to the topics.

In [12]:
import pandas as pd
import pickle
import numpy as np
from sklearn.metrics import pairwise_distances
from pymongo import MongoClient


In [4]:
topic_prob = pd.read_csv('../data/topic_prob.csv', index_col=0)

In [9]:
topic_prob = topic_prob.groupby(['url', 'index'], as_index=False).sum()

In [17]:
client = MongoClient('ec2-34-198-179-91.compute-1.amazonaws.com', 27017)
db = client.fletcher
rev_col = db.rtr_reviews

In [18]:
df_rev = pd.DataFrame(list(rev_col.find({}, {'review': 1, '_id': 0} )))

In [81]:
df = topic_prob.join(df_rev)
# df = pd.read_csv('../data/comment_search.csv')

In [82]:
def get_comments(df, url, columns, n):
    cur_df = df[df.url == url]
    cur_df = cur_df.loc[:, columns]
    cur_df['dist'] = pairwise_distances(cur_df, [1] * len(columns))
    return list(df[df.index.isin(cur_df.sort_values('dist').head(n).index)].review.values)
    

In [112]:

def get_comments(df_comment, url, columns, n):
    cur_df = df_comment[df_comment.url == url]
    cur_df = cur_df.loc[:, columns]
    cur_df['dist'] = pairwise_distances(cur_df, [1] * len(columns), metric='cosine')
    cur_df = cur_df.sort_values('dist').head(20)
    cur_df = df_comment[df_comment.index.isin(cur_df.index)].sort_values('polarity', ascending=False).head(n)
    return list(cur_df.review.values)


In [90]:
url =  'https://www.renttherunway.com/shop/designers/mark__james_by_badgley_mischka/mini_sequin_pixie_dress'

In [113]:
get_comments(df, url, ['length'], 10)

/Users/josephinetirtanata/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


['I got so many compliments on this dress!  The print is unique.  It fit like a glove and was very comfortable.',
 "This dress fit perfectly. I am an athletic, large-chested 130lbs and the size 4 looked like it was made for me. I rented this dress as a back up/rehearsal dinner option but ended up loving it so much I wore it to the wedding. I received many compliments on it. The fabric and lining are high quality and didn't wrinkle or fall out of shape through the 10 hours I wore it. I also appreciated that the arm holes are small enough so that you don't see your bra, but large enough to be comfortable. Very happy with this dress!",
 'This is a beautiful dress -- lovely navy blue color with really subtle cheetah print and gorgeous silver detail on the bottom. I rented sizes 6 and 8 and it turns out I was in between sizes. The 8 was too baggy on top and the arm holes were too large, so I went with the 6 which was very tight across the bust and created an unfortunate "uni-boob" effect. U

In [121]:
df.sort_values('length', ascending=False).review.values[:5]

array([ 'This is a truly beautiful and classy dress. The open back gave it the perfect touch of sexiness. The lace has a weight to the bottom that gives a good-quality feel to it, and it flows beautifully as you walk. I am 5\'7" 125lbs 32A and usually wear a size 2 dress. I wore the 2L with 3" heels and it was still an inch or two too long for my liking. My heels caught on the lace at the bottom when dancing, so I had to hold it up a little in order to not fall in my face.. As other reviews have noted, the top was loose and seemed to be a little stretched out but it was definitely still wearable. I wore an adhesive bra with this dress, which worked well until I worked up a little sweat. That plus the humidity had me worried that the bra cups would fall right off. Overall I thought this dress was gorgeous and photographed well. I received many compliments throughout the night. I gave this dress four out of five stars because it was a little bit higher maintenance than I would have liked